# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started. Once you have completed the code you can download the notebook for making a submission.

In [1]:
import numpy as np
import os
from imageio import imread
from skimage.transform import resize
import datetime
import os
import imageio

import cv2
import matplotlib.pyplot as plt
%matplotlib inline

We set the random seed so that the results don't vary drastically.

In [3]:
np.random.seed(30)
import random as rn
rn.seed(30)
from tensorflow import keras
import tensorflow as tf
tf.random.set_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

**data path: /home/datasets/Project_data**

In [4]:
train_doc = np.random.permutation(open('/home/datasets/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('/home/datasets/Project_data/val.csv').readlines())
batch_size = 51
train_doc

array(['WIN_20180926_16_54_08_Pro_Right_Swipe_new;Right_Swipe_new;1\n',
       'WIN_20180925_18_02_58_Pro_Thumbs_Down_new;Thumbs_Down_new;3\n',
       'WIN_20180925_17_33_08_Pro_Left_Swipe_new;Left_Swipe_new;0\n',
       'WIN_20180925_17_51_17_Pro_Thumbs_Up_new;Thumbs_Up_new;4\n',
       'WIN_20180926_17_17_35_Pro_Left_Swipe_new;Left_Swipe_new;0\n',
       'WIN_20180926_17_30_47_Pro_Thumbs_Up_new;Thumbs_Up_new;4\n',
       'WIN_20180926_17_29_23_Pro_Stop_new;Stop_new;2\n',
       'WIN_20180926_17_12_26_Pro_Thumbs_Up_new;Thumbs_Up_new;4\n',
       'WIN_20180926_17_29_01_Pro_Stop_new;Stop_new;2\n',
       'WIN_20180907_16_32_11_Pro_Stop Gesture_new;Stop Gesture_new;2\n',
       'WIN_20180926_17_06_01_Pro_Stop_new;Stop_new;2\n',
       'WIN_20180907_16_52_18_Pro_Stop Gesture_new;Stop Gesture_new;2\n',
       'WIN_20180907_16_39_19_Pro_Left Swipe_new_Left Swipe_new;Left Swipe_new_Left Swipe_new;0\n',
       'WIN_20180925_17_19_16_Pro_Thumbs_Up_new;Thumbs_Up_new;4\n',
       'WIN_20180907_1

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [5]:
def generator(source_path, folder_list, batch_size,frame_count, img_y, img_z):
    #print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = np.round(np.linspace(0,29,frame_count)).astype(int)#create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(t)//batch_size # calculate the number of batches

        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,frame_count, img_y, img_z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output

            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                                       
                    # Cropped image of above dimension 
                    
                    #image = image.crop((0, 0, 120, 120))
                    image_resized=resize(image,(img_y, img_z,3))
                    
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    batch_data[folder,idx,:,:,0] =(image_resized[:,:,0])/255#normalise and feed in the image
                    batch_data[folder,idx,:,:,1] =(image_resized[:,:,1])/255#normalise and feed in the image
                    batch_data[folder,idx,:,:,2] =(image_resized[:,:,2])/255#normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        remain_seq=len(t)%batch_size
        if (remain_seq != 0):
            batch_size = remain_seq if remain_seq else batch_size
        
            for batch in range(num_batches): # we iterate over the number of batches
                batch_data = np.zeros((batch_size,frame_count, img_y, img_z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
                batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output

                for folder in range(batch_size): # iterate over the batch_size
                    imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder

                    for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                        image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)

                        # Cropped image of above dimension 

                        #image = image.crop((0, 0, 120, 120))
                        image_resized=resize(image,(img_y, img_z,3))


                        #crop the images and resize them. Note that the images are of 2 different shape 
                        #and the conv3D will throw error if the inputs in a batch have different shapes

                        batch_data[folder,idx,:,:,0] =(image_resized[:,:,0])/255#normalise and feed in the image
                        batch_data[folder,idx,:,:,1] =(image_resized[:,:,1])/255#normalise and feed in the image
                        batch_data[folder,idx,:,:,2] =(image_resized[:,:,2])/255#normalise and feed in the image

                    batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                yield batch_data, batch_labels 

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [6]:
curr_dt_time = datetime.datetime.now()
train_path = '/home/datasets/Project_data/train'
val_path = '/home/datasets/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 15 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 15


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

#### Base Conv3D Model

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Dropout, Flatten, BatchNormalization, Activation, Conv3D, MaxPooling3D
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import optimizers

#write your model here
def define_model(Input_shape):
    model = Sequential()
    model.add(Conv3D(16, (3,3,3), padding='same',input_shape=Input_shape))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    model.add(Conv3D(32, (3,3,3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling3D(pool_size=(2,2,2)))

    model.add(Conv3D(64, (3,3,3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling3D(pool_size=(2,2,2)))
 
    model.add(Conv3D(128, (3,3,3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling3D(pool_size=(2,2,2)))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))

    model.add(Flatten())           
    model.add(Dense(5,activation='softmax'))
    
    optimiser = optimizers.Adam()#write your optimizer
    model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    
    return model

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [8]:
frame_count=10
img_y =100
img_z =100
Input_shape = (frame_count, img_y, img_z, 3)
model_1 = define_model(Input_shape)
print (model_1.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 10, 100, 100, 16)  1312      
_________________________________________________________________
batch_normalization (BatchNo (None, 10, 100, 100, 16)  64        
_________________________________________________________________
activation (Activation)      (None, 10, 100, 100, 16)  0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 10, 100, 100, 32)  13856     
_________________________________________________________________
activation_1 (Activation)    (None, 10, 100, 100, 32)  0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 10, 100, 100, 32)  128       
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 5, 50, 50, 32)     0

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [9]:
def model_train_fit(model,train_path,train_doc,val_path, val_doc,batch_size,frame_count,img_y,img_z,num_train_sequences,num_val_sequences,num_epochs):
    
    train_generator = generator(train_path, train_doc, batch_size, frame_count, img_y, img_z)
    val_generator = generator(val_path, val_doc, batch_size,frame_count, img_y, img_z)

    model_name = 'Gesture_recog' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'

    if not os.path.exists(model_name):
        os.mkdir(model_name)

    filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

    checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

    LR = ReduceLROnPlateau(monitor = "val_loss", factor = 0.1, patience = 10,
      verbose = 0, mode = "auto", epsilon = 1e-04, cooldown = 0,
      min_lr = 0)
    callbacks_list = [checkpoint, LR]

    #The `steps_per_epoch` and `validation_steps` are used by `fit` method to decide the number of next() calls it need to make.

    if (num_train_sequences%batch_size) == 0:
        steps_per_epoch = int(num_train_sequences/batch_size)
    else:
        steps_per_epoch = (num_train_sequences//batch_size) + 1

    if (num_val_sequences%batch_size) == 0:
        validation_steps = int(num_val_sequences/batch_size)
    else:
        validation_steps = (num_val_sequences//batch_size) + 1

    #Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

    model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                        callbacks=callbacks_list, validation_data=val_generator, 
                        validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

In [89]:
num_epochs = 10
model_train_fit(model_1,train_path,train_doc,val_path, val_doc,batch_size,
                      frame_count,img_y,img_z,num_train_sequences,num_val_sequences,num_epochs)

Epoch 1/10
13/13 [==============================] - ETA: 0s - loss: 19.8959 - categorical_accuracy: 0.2881
Epoch 00001: saving model to Gesture_recog_2021-09-2900_25_48.938246/model-00001-19.89591-0.28808-61.31864-0.33000.h5
13/13 [==============================] - 36s 3s/step - loss: 19.8959 - categorical_accuracy: 0.2881 - val_loss: 61.3186 - val_categorical_accuracy: 0.3300
Epoch 2/10
13/13 [==============================] - ETA: 0s - loss: 5.1379 - categorical_accuracy: 0.4087
Epoch 00002: saving model to Gesture_recog_2021-09-2900_25_48.938246/model-00002-5.13794-0.40875-19.37234-0.17647.h5
13/13 [==============================] - 35s 3s/step - loss: 5.1379 - categorical_accuracy: 0.4087 - val_loss: 19.3723 - val_categorical_accuracy: 0.1765
Epoch 3/10
13/13 [==============================] - ETA: 0s - loss: 1.4974 - categorical_accuracy: 0.5189
Epoch 00003: saving model to Gesture_recog_2021-09-2900_25_48.938246/model-00003-1.49741-0.51885-2.68782-0.25000.h5
13/13 [==============

Observation: Model is Overfitting, Lets increase the epochs, to increase the overall accuracy

#### Conv3D Model-2

The `steps_per_epoch` and `validation_steps` are used by `fit` method to decide the number of next() calls it need to make.

In [90]:
num_epochs = 15
model_train_fit(model_1,train_path,train_doc,val_path, val_doc,batch_size,
                      frame_count,img_y,img_z,num_train_sequences,num_val_sequences,num_epochs)

Epoch 1/15
13/13 [==============================] - ETA: 0s - loss: 0.6377 - categorical_accuracy: 0.7270
Epoch 00001: saving model to Gesture_recog_2021-09-2900_25_48.938246/model-00001-0.63772-0.72700-4.83760-0.14000.h5
13/13 [==============================] - 39s 3s/step - loss: 0.6377 - categorical_accuracy: 0.7270 - val_loss: 4.8376 - val_categorical_accuracy: 0.1400
Epoch 2/15
13/13 [==============================] - ETA: 0s - loss: 0.6055 - categorical_accuracy: 0.7436
Epoch 00002: saving model to Gesture_recog_2021-09-2900_25_48.938246/model-00002-0.60555-0.74359-4.08749-0.27451.h5
13/13 [==============================] - 33s 3s/step - loss: 0.6055 - categorical_accuracy: 0.7436 - val_loss: 4.0875 - val_categorical_accuracy: 0.2745
Epoch 3/15
13/13 [==============================] - ETA: 0s - loss: 0.5746 - categorical_accuracy: 0.7692
Epoch 00003: saving model to Gesture_recog_2021-09-2900_25_48.938246/model-00003-0.57456-0.76923-2.37545-0.50000.h5
13/13 [=====================

Observation:Still overfitting, Next increasing the frame count & Image resolution. As we see that the accuracy can be further improved and reduce over fitting

#### Conv3D Model-3

In [99]:
frame_count=16
img_y =120
img_z =120
Input_shape = (frame_count, img_y, img_z, 3)
model_1 = define_model(Input_shape)
print (model_1.summary())

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_33 (Conv3D)           (None, 16, 120, 120, 16)  1312      
_________________________________________________________________
batch_normalization_29 (Batc (None, 16, 120, 120, 16)  64        
_________________________________________________________________
activation_45 (Activation)   (None, 16, 120, 120, 16)  0         
_________________________________________________________________
conv3d_34 (Conv3D)           (None, 16, 120, 120, 32)  13856     
_________________________________________________________________
activation_46 (Activation)   (None, 16, 120, 120, 32)  0         
_________________________________________________________________
batch_normalization_30 (Batc (None, 16, 120, 120, 32)  128       
_________________________________________________________________
max_pooling3d_18 (MaxPooling (None, 8, 60, 60, 32)    

In [100]:
num_epochs = 15
model_train_fit(model_1,train_path,train_doc,val_path, val_doc,batch_size,
                      frame_count,img_y,img_z,num_train_sequences,num_val_sequences,num_epochs)

Epoch 1/15
13/13 [==============================] - ETA: 0s - loss: 52.7714 - categorical_accuracy: 0.2851
Epoch 00001: saving model to Gesture_recog_2021-09-2900_25_48.938246/model-00001-52.77138-0.28507-16.84296-0.31000.h5
13/13 [==============================] - 89s 7s/step - loss: 52.7714 - categorical_accuracy: 0.2851 - val_loss: 16.8430 - val_categorical_accuracy: 0.3100
Epoch 2/15
13/13 [==============================] - ETA: 0s - loss: 8.3455 - categorical_accuracy: 0.4600
Epoch 00002: saving model to Gesture_recog_2021-09-2900_25_48.938246/model-00002-8.34555-0.46003-3.94958-0.21569.h5
13/13 [==============================] - 74s 6s/step - loss: 8.3455 - categorical_accuracy: 0.4600 - val_loss: 3.9496 - val_categorical_accuracy: 0.2157
Epoch 3/15
13/13 [==============================] - ETA: 0s - loss: 1.4670 - categorical_accuracy: 0.5158
Epoch 00003: saving model to Gesture_recog_2021-09-2900_25_48.938246/model-00003-1.46699-0.51584-2.33466-0.00000.h5
13/13 [================

Observation:Model is still Overfitting. So, increasing the resolution & reducing the batch size resp

#### Conv3D Model-4

In [101]:
num_epochs = 20
model_train_fit(model_1,train_path,train_doc,val_path, val_doc,batch_size,
                      frame_count,img_y,img_z,num_train_sequences,num_val_sequences,num_epochs)

Epoch 1/20
13/13 [==============================] - ETA: 0s - loss: 0.5780 - categorical_accuracy: 0.7617
Epoch 00001: saving model to Gesture_recog_2021-09-2900_25_48.938246/model-00001-0.57803-0.76169-6.54973-0.26000.h5
13/13 [==============================] - 84s 6s/step - loss: 0.5780 - categorical_accuracy: 0.7617 - val_loss: 6.5497 - val_categorical_accuracy: 0.2600
Epoch 2/20
13/13 [==============================] - ETA: 0s - loss: 0.5953 - categorical_accuracy: 0.7345
Epoch 00002: saving model to Gesture_recog_2021-09-2900_25_48.938246/model-00002-0.59527-0.73454-8.83178-0.15686.h5
13/13 [==============================] - 71s 5s/step - loss: 0.5953 - categorical_accuracy: 0.7345 - val_loss: 8.8318 - val_categorical_accuracy: 0.1569
Epoch 3/20
13/13 [==============================] - ETA: 0s - loss: 0.5574 - categorical_accuracy: 0.7466
Epoch 00003: saving model to Gesture_recog_2021-09-2900_25_48.938246/model-00003-0.55744-0.74661-3.92521-0.50000.h5
13/13 [=====================

Observation:Seen improvement but sill the accuracy can improve with good fit. Next trying ConvLSTM to see if we get better results

#### Conv2D RNN + LSTM Model-5

In [10]:
from tensorflow.keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout, LSTM, Bidirectional, Conv2D, MaxPooling2D

def define_conv2d_model(Input_shape):
    model = Sequential()
    model.add(TimeDistributed(Conv2D(16, (3, 3) , padding='same', activation='relu'),
                              input_shape=Input_shape))
    model.add(TimeDistributed(BatchNormalization()))
    model.add(TimeDistributed(MaxPooling2D((2, 2))))

    model.add(TimeDistributed(Conv2D(32, (3, 3) , padding='same', activation='relu')))
    model.add(TimeDistributed(BatchNormalization()))
    model.add(TimeDistributed(MaxPooling2D((2, 2))))

    model.add(TimeDistributed(Conv2D(64, (3, 3) , padding='same', activation='relu')))
    model.add(TimeDistributed(BatchNormalization()))
    model.add(TimeDistributed(MaxPooling2D((2, 2))))

    model.add(TimeDistributed(Conv2D(128, (3, 3) , padding='same', activation='relu')))
    model.add(TimeDistributed(BatchNormalization()))
    model.add(TimeDistributed(MaxPooling2D((2, 2))))

    model.add(TimeDistributed(Conv2D(256, (3, 3) , padding='same', activation='relu')))
    model.add(TimeDistributed(BatchNormalization()))
    model.add(TimeDistributed(MaxPooling2D((2, 2))))

    model.add(TimeDistributed(Flatten()))

    model.add(Dropout(0.25))
    model.add(LSTM(128))
    model.add(Dense(5, activation='softmax'))
    
    optimiser = optimizers.Adam()#write your optimizer
    model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    
    return model

In [115]:
frame_count=16
batch_size= 20
img_y =120
img_z =120
Input_shape = (frame_count, img_y, img_z, 3)
convd2_model_1 = define_conv2d_model(Input_shape)
print (convd2_model_1.summary())

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_77 (TimeDis (None, 16, 120, 120, 16)  448       
_________________________________________________________________
time_distributed_78 (TimeDis (None, 16, 120, 120, 16)  64        
_________________________________________________________________
time_distributed_79 (TimeDis (None, 16, 60, 60, 16)    0         
_________________________________________________________________
time_distributed_80 (TimeDis (None, 16, 60, 60, 32)    4640      
_________________________________________________________________
time_distributed_81 (TimeDis (None, 16, 60, 60, 32)    128       
_________________________________________________________________
time_distributed_82 (TimeDis (None, 16, 30, 30, 32)    0         
_________________________________________________________________
time_distributed_83 (TimeDis (None, 16, 30, 30, 64)  

In [116]:
num_epochs = 20
model_train_fit(convd2_model_1,train_path,train_doc,val_path, val_doc,batch_size,
                      frame_count,img_y,img_z,num_train_sequences,num_val_sequences,num_epochs)

Epoch 1/20
34/34 [==============================] - ETA: 0s - loss: 1.1396 - categorical_accuracy: 0.5566
Epoch 00001: saving model to Gesture_recog_2021-09-2900_25_48.938246/model-00001-1.13959-0.55656-1.62830-0.22000.h5
34/34 [==============================] - 64s 2s/step - loss: 1.1396 - categorical_accuracy: 0.5566 - val_loss: 1.6283 - val_categorical_accuracy: 0.2200
Epoch 2/20
34/34 [==============================] - ETA: 0s - loss: 1.1642 - categorical_accuracy: 0.5392
Epoch 00002: saving model to Gesture_recog_2021-09-2900_25_48.938246/model-00002-1.16422-0.53922-1.90190-0.22000.h5
34/34 [==============================] - 21s 623ms/step - loss: 1.1642 - categorical_accuracy: 0.5392 - val_loss: 1.9019 - val_categorical_accuracy: 0.2200
Epoch 3/20
34/34 [==============================] - ETA: 0s - loss: 1.0847 - categorical_accuracy: 0.5686
Epoch 00003: saving model to Gesture_recog_2021-09-2900_25_48.938246/model-00003-1.08467-0.56863-2.48171-0.18000.h5
34/34 [==================

Observation: Accuracy increased significantly, as we see that the accuracy can be further improved. Increasing the number of epochs

#### Conv2D RNN + LSTM Model-6

In [11]:
#Finding optimal frame_count and hyper parameters
frame_count=16
batch_size=10
img_y =120
img_z =120
Input_shape = (frame_count, img_y, img_z, 3)
convd2_model_1 = define_conv2d_model(Input_shape)
print (convd2_model_1.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri (None, 16, 120, 120, 16)  448       
_________________________________________________________________
time_distributed_1 (TimeDist (None, 16, 120, 120, 16)  64        
_________________________________________________________________
time_distributed_2 (TimeDist (None, 16, 60, 60, 16)    0         
_________________________________________________________________
time_distributed_3 (TimeDist (None, 16, 60, 60, 32)    4640      
_________________________________________________________________
time_distributed_4 (TimeDist (None, 16, 60, 60, 32)    128       
_________________________________________________________________
time_distributed_5 (TimeDist (None, 16, 30, 30, 32)    0         
_________________________________________________________________
time_distributed_6 (TimeDist (None, 16, 30, 30, 64)    1

In [12]:
num_epochs = 25
model_train_fit(convd2_model_1,train_path,train_doc,val_path, val_doc,batch_size,
                      frame_count,img_y,img_z,num_train_sequences,num_val_sequences,num_epochs)

Epoch 1/25
67/67 [==============================] - ETA: 0s - loss: 1.1999 - categorical_accuracy: 0.5128
Epoch 00001: saving model to Gesture_recog_2021-09-2910_22_53.093090/model-00001-1.19993-0.51282-2.33615-0.21000.h5
67/67 [==============================] - 132s 2s/step - loss: 1.1999 - categorical_accuracy: 0.5128 - val_loss: 2.3362 - val_categorical_accuracy: 0.2100
Epoch 2/25
67/67 [==============================] - ETA: 0s - loss: 1.1285 - categorical_accuracy: 0.5622
Epoch 00002: saving model to Gesture_recog_2021-09-2910_22_53.093090/model-00002-1.12852-0.56219-2.00321-0.21000.h5
67/67 [==============================] - 51s 762ms/step - loss: 1.1285 - categorical_accuracy: 0.5622 - val_loss: 2.0032 - val_categorical_accuracy: 0.2100
Epoch 3/25
67/67 [==============================] - ETA: 0s - loss: 1.1462 - categorical_accuracy: 0.5323
Epoch 00003: saving model to Gesture_recog_2021-09-2910_22_53.093090/model-00003-1.14617-0.53234-2.45330-0.23000.h5
67/67 [=================

Observation: Not much improvement in the validation accuracy, next trying increase with the increase in image resolution and number of epochs
    
#### Conv2D RNN + LSTM Model-7

In [11]:
frame_count=16
batch_size= 10
img_y =160
img_z =160
Input_shape = (frame_count, img_y, img_z, 3)
convd2_model_1 = define_conv2d_model(Input_shape)
print (convd2_model_1.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri (None, 16, 160, 160, 16)  448       
_________________________________________________________________
time_distributed_1 (TimeDist (None, 16, 160, 160, 16)  64        
_________________________________________________________________
time_distributed_2 (TimeDist (None, 16, 80, 80, 16)    0         
_________________________________________________________________
time_distributed_3 (TimeDist (None, 16, 80, 80, 32)    4640      
_________________________________________________________________
time_distributed_4 (TimeDist (None, 16, 80, 80, 32)    128       
_________________________________________________________________
time_distributed_5 (TimeDist (None, 16, 40, 40, 32)    0         
_________________________________________________________________
time_distributed_6 (TimeDist (None, 16, 40, 40, 64)   

In [12]:
num_epochs = 25
model_train_fit(convd2_model_1,train_path,train_doc,val_path, val_doc,batch_size,
                      frame_count,img_y,img_z,num_train_sequences,num_val_sequences,num_epochs)

Epoch 1/25
67/67 [==============================] - ETA: 0s - loss: 1.2867 - categorical_accuracy: 0.5128
Epoch 00001: saving model to Gesture_recog_2021-09-2911_36_38.908657/model-00001-1.28670-0.51282-1.85981-0.13000.h5
67/67 [==============================] - 122s 2s/step - loss: 1.2867 - categorical_accuracy: 0.5128 - val_loss: 1.8598 - val_categorical_accuracy: 0.1300
Epoch 2/25
67/67 [==============================] - ETA: 0s - loss: 1.2839 - categorical_accuracy: 0.4527
Epoch 00002: saving model to Gesture_recog_2021-09-2911_36_38.908657/model-00002-1.28385-0.45274-1.80355-0.23000.h5
67/67 [==============================] - 41s 617ms/step - loss: 1.2839 - categorical_accuracy: 0.4527 - val_loss: 1.8036 - val_categorical_accuracy: 0.2300
Epoch 3/25
67/67 [==============================] - ETA: 0s - loss: 1.3096 - categorical_accuracy: 0.4378
Epoch 00003: saving model to Gesture_recog_2021-09-2911_36_38.908657/model-00003-1.30964-0.43781-1.81042-0.30000.h5
67/67 [=================

Observation: Accuracy decreased, now trying with ConvGRU
    
#### Conv2D RNN + GRU Model-8

In [ ]:
from tensorflow.keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout, LSTM, Bidirectional, Conv2D, MaxPooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Dropout, Flatten, BatchNormalization, Activation, Conv3D, MaxPooling3D
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import optimizers


def define_conv2d_GRU_model(Input_shape, gru_cells=64, dense_neurons=64):
    model = Sequential()
    model.add(TimeDistributed(Conv2D(16, (3, 3) , padding='same', activation='relu'),
                              input_shape=Input_shape))
    model.add(TimeDistributed(BatchNormalization()))
    model.add(TimeDistributed(MaxPooling2D((2, 2))))

    model.add(TimeDistributed(Conv2D(32, (3, 3) , padding='same', activation='relu')))
    model.add(TimeDistributed(BatchNormalization()))
    model.add(TimeDistributed(MaxPooling2D((2, 2))))

    model.add(TimeDistributed(Conv2D(64, (3, 3) , padding='same', activation='relu')))
    model.add(TimeDistributed(BatchNormalization()))
    model.add(TimeDistributed(MaxPooling2D((2, 2))))

    model.add(TimeDistributed(Conv2D(128, (3, 3) , padding='same', activation='relu')))
    model.add(TimeDistributed(BatchNormalization()))
    model.add(TimeDistributed(MaxPooling2D((2, 2))))

    model.add(TimeDistributed(Conv2D(256, (3, 3) , padding='same', activation='relu')))
    model.add(TimeDistributed(BatchNormalization()))
    model.add(TimeDistributed(MaxPooling2D((2, 2))))

    model.add(TimeDistributed(Flatten()))

    model.add(GRU(gru_cells))
    model.add(Dropout(.25))
        
    model.add(Dense(dense_neurons,activation='relu'))
    model.add(Dropout(.25))
    
    model.add(Dense(5, activation='softmax'))

    
    optimiser = optimizers.Adam()#write your optimizer
    model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    
    return model

In [10]:
frame_count=16
batch_size= 20
img_y =120
img_z =120
Input_shape = (frame_count, img_y, img_z, 3)
convd2_gru_model_1 = define_conv2d_GRU_model(Input_shape)
print (convd2_gru_model_1.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri (None, 16, 120, 120, 16)  448       
_________________________________________________________________
time_distributed_1 (TimeDist (None, 16, 120, 120, 16)  64        
_________________________________________________________________
time_distributed_2 (TimeDist (None, 16, 60, 60, 16)    0         
_________________________________________________________________
time_distributed_3 (TimeDist (None, 16, 60, 60, 32)    4640      
_________________________________________________________________
time_distributed_4 (TimeDist (None, 16, 60, 60, 32)    128       
_________________________________________________________________
time_distributed_5 (TimeDist (None, 16, 30, 30, 32)    0         
_________________________________________________________________
time_distributed_6 (TimeDist (None, 16, 30, 30, 64)    1

In [11]:
num_epochs = 20
model_train_fit(convd2_gru_model_1,train_path,train_doc,val_path, val_doc,batch_size,
                      frame_count,img_y,img_z,num_train_sequences,num_val_sequences,num_epochs)

Epoch 1/20
34/34 [==============================] - ETA: 0s - loss: 1.4810 - categorical_accuracy: 0.3756
Epoch 00001: saving model to Gesture_recog_2021-09-2909_03_15.199507/model-00001-1.48101-0.37557-2.11356-0.23000.h5
34/34 [==============================] - 137s 4s/step - loss: 1.4810 - categorical_accuracy: 0.3756 - val_loss: 2.1136 - val_categorical_accuracy: 0.2300
Epoch 2/20
34/34 [==============================] - ETA: 0s - loss: 1.3596 - categorical_accuracy: 0.3627
Epoch 00002: saving model to Gesture_recog_2021-09-2909_03_15.199507/model-00002-1.35960-0.36275-1.92307-0.26000.h5
34/34 [==============================] - 38s 1s/step - loss: 1.3596 - categorical_accuracy: 0.3627 - val_loss: 1.9231 - val_categorical_accuracy: 0.2600
Epoch 3/20
34/34 [==============================] - ETA: 0s - loss: 1.5214 - categorical_accuracy: 0.3627
Epoch 00003: saving model to Gesture_recog_2021-09-2909_03_15.199507/model-00003-1.52137-0.36275-2.09951-0.21000.h5
34/34 [====================

Observation: As we see no good performance in validation accuracy, so increasing number of epochs and decreasing batch size
    
#### Conv2D RNN + GRU Model-9

In [12]:
frame_count=16
batch_size= 10
img_y =120
img_z =120
Input_shape = (frame_count, img_y, img_z, 3)
convd2_gru_model_1 = define_conv2d_GRU_model(Input_shape)
print (convd2_gru_model_1.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_16 (TimeDis (None, 16, 120, 120, 16)  448       
_________________________________________________________________
time_distributed_17 (TimeDis (None, 16, 120, 120, 16)  64        
_________________________________________________________________
time_distributed_18 (TimeDis (None, 16, 60, 60, 16)    0         
_________________________________________________________________
time_distributed_19 (TimeDis (None, 16, 60, 60, 32)    4640      
_________________________________________________________________
time_distributed_20 (TimeDis (None, 16, 60, 60, 32)    128       
_________________________________________________________________
time_distributed_21 (TimeDis (None, 16, 30, 30, 32)    0         
_________________________________________________________________
time_distributed_22 (TimeDis (None, 16, 30, 30, 64)   

In [13]:
num_epochs = 25
model_train_fit(convd2_gru_model_1,train_path,train_doc,val_path, val_doc,batch_size,
                      frame_count,img_y,img_z,num_train_sequences,num_val_sequences,num_epochs)

Epoch 1/25
67/67 [==============================] - ETA: 0s - loss: 1.5155 - categorical_accuracy: 0.3258
Epoch 00001: saving model to Gesture_recog_2021-09-2909_03_15.199507/model-00001-1.51548-0.32579-1.78018-0.21000.h5
67/67 [==============================] - 134s 2s/step - loss: 1.5155 - categorical_accuracy: 0.3258 - val_loss: 1.7802 - val_categorical_accuracy: 0.2100
Epoch 2/25
67/67 [==============================] - ETA: 0s - loss: 1.4242 - categorical_accuracy: 0.3831
Epoch 00002: saving model to Gesture_recog_2021-09-2909_03_15.199507/model-00002-1.42416-0.38308-1.79421-0.23000.h5
67/67 [==============================] - 59s 877ms/step - loss: 1.4242 - categorical_accuracy: 0.3831 - val_loss: 1.7942 - val_categorical_accuracy: 0.2300
Epoch 3/25
67/67 [==============================] - ETA: 0s - loss: 1.4745 - categorical_accuracy: 0.3731
Epoch 00003: saving model to Gesture_recog_2021-09-2909_03_15.199507/model-00003-1.47455-0.37313-1.99162-0.19000.h5
67/67 [=================

Observation: As we see no good performance in validation accuracy switching to GRU+MobileNet
    
#### GRU + MobileNet Model-10

In [13]:
from tensorflow.keras.applications import mobilenet

mobilenet_transfer = mobilenet.MobileNet(weights='imagenet', include_top=False)

def model_gru_mobilenet(Input_shape,gru_cells=64,dense_neurons=64,dropout=0.25):
        
    model = Sequential()
    model.add(TimeDistributed(mobilenet_transfer,input_shape=Input_shape))
 
        
    model.add(TimeDistributed(BatchNormalization()))
    model.add(TimeDistributed(MaxPooling2D((2, 2))))
    model.add(TimeDistributed(Flatten()))

    model.add(GRU(gru_cells))
    model.add(Dropout(dropout))
        
    model.add(Dense(dense_neurons,activation='relu'))
    model.add(Dropout(dropout))
        
    model.add(Dense(5, activation='softmax'))
        
        
    optimiser = optimizers.Adam()
    model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    return model

In [15]:
frame_count=16
img_y =120
img_z =120
batch_size=5
num_epochs=20
Input_shape = (frame_count, img_y, img_z, 3)
model_mobilenet= model_gru_mobilenet(Input_shape)
print (model_mobilenet.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_17 (TimeDis (None, 16, 3, 3, 1024)    3228864   
_________________________________________________________________
time_distributed_18 (TimeDis (None, 16, 3, 3, 1024)    4096      
_________________________________________________________________
time_distributed_19 (TimeDis (None, 16, 1, 1, 1024)    0         
_________________________________________________________________
time_distributed_20 (TimeDis (None, 16, 1024)          0         
_________________________________________________________________
gru (GRU)                    (None, 64)                209280    
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 64)               

In [16]:
model_train_fit(model_mobilenet,train_path,train_doc,val_path, val_doc,batch_size,frame_count,img_y,img_z,num_train_sequences,num_val_sequences,num_epochs)

Epoch 1/20
133/133 [==============================] - ETA: 0s - loss: 1.2338 - categorical_accuracy: 0.4751
Epoch 00001: saving model to Gesture_recog_2021-09-2911_36_38.908657/model-00001-1.23378-0.47511-0.80729-0.71000.h5
133/133 [==============================] - 78s 584ms/step - loss: 1.2338 - categorical_accuracy: 0.4751 - val_loss: 0.8073 - val_categorical_accuracy: 0.7100
Epoch 2/20
133/133 [==============================] - ETA: 0s - loss: 1.0264 - categorical_accuracy: 0.6115
Epoch 00002: saving model to Gesture_recog_2021-09-2911_36_38.908657/model-00002-1.02636-0.61153-1.12036-0.60000.h5
133/133 [==============================] - 53s 395ms/step - loss: 1.0264 - categorical_accuracy: 0.6115 - val_loss: 1.1204 - val_categorical_accuracy: 0.6000
Epoch 3/20
133/133 [==============================] - ETA: 0s - loss: 0.9383 - categorical_accuracy: 0.6416
Epoch 00003: saving model to Gesture_recog_2021-09-2911_36_38.908657/model-00003-0.93833-0.64160-0.78088-0.70000.h5
133/133 [===

Observation: We can improve the accuracy by tweaking the parameters like batch size & Epochs
    
#### GRU + MobileNet Model-11

In [18]:
frame_count=16
img_y =120
img_z =120
batch_size=10
num_epochs=25
Input_shape = (frame_count, img_y, img_z, 3)
model_mobilenet= model_gru_mobilenet(Input_shape)
print (model_mobilenet.summary())

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_22 (TimeDis (None, 16, 3, 3, 1024)    3228864   
_________________________________________________________________
time_distributed_23 (TimeDis (None, 16, 3, 3, 1024)    4096      
_________________________________________________________________
time_distributed_24 (TimeDis (None, 16, 1, 1, 1024)    0         
_________________________________________________________________
time_distributed_25 (TimeDis (None, 16, 1024)          0         
_________________________________________________________________
gru_1 (GRU)                  (None, 64)                209280    
_________________________________________________________________
dropout_4 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 64)               

In [19]:
model_train_fit(model_mobilenet,train_path,train_doc,val_path, val_doc,batch_size,frame_count,img_y,img_z,num_train_sequences,num_val_sequences,num_epochs)

Epoch 1/25
67/67 [==============================] - ETA: 0s - loss: 0.5866 - categorical_accuracy: 0.8009
Epoch 00001: saving model to Gesture_recog_2021-09-2911_36_38.908657/model-00001-0.58658-0.80090-0.73523-0.76000.h5
67/67 [==============================] - 80s 1s/step - loss: 0.5866 - categorical_accuracy: 0.8009 - val_loss: 0.7352 - val_categorical_accuracy: 0.7600
Epoch 2/25
67/67 [==============================] - ETA: 0s - loss: 0.6139 - categorical_accuracy: 0.8010
Epoch 00002: saving model to Gesture_recog_2021-09-2911_36_38.908657/model-00002-0.61389-0.80100-0.84249-0.73000.h5
67/67 [==============================] - 36s 532ms/step - loss: 0.6139 - categorical_accuracy: 0.8010 - val_loss: 0.8425 - val_categorical_accuracy: 0.7300
Epoch 3/25
67/67 [==============================] - ETA: 0s - loss: 0.7040 - categorical_accuracy: 0.7662
Epoch 00003: saving model to Gesture_recog_2021-09-2911_36_38.908657/model-00003-0.70397-0.76617-0.77020-0.76000.h5
67/67 [==================

Observation: This seems to be the best fit accuracy so far.

Final Observation:
Post running all the experiments we see that Conv3D can be further improved as last model 4 has 0.78 & 0.50 accuracy which is a clear over fitting case. But on using GRU + MobileNet we could handle such over-fitting and get more stable model.
